In [1]:
!pip install -q xgboost

In [2]:
import joblib
import numpy as np 
import pandas as pd 
import sklearn
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from joblib import dump, load
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('dataset.csv')
df

,Unnamed: 0,Index,Address,FLAG,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,...,ERC20 min val sent,ERC20 max val sent,ERC20 avg val sent,ERC20 min val sent contract,ERC20 max val sent contract,ERC20 avg val sent contract,ERC20 uniq sent token name,ERC20 uniq rec token name,ERC20 most sent token type,ERC20_most_rec_token_type
0,0,1,0x00009277775ac7d0d59eaad8fee3d10ac6c805e8,0,844.26,1093.71,704785.63,721,89,0,...,0.000000,1.683100e+07,271779.920000,0.0,0.0,0.0,39.0,57.0,Cofoundit,Numeraire
1,1,2,0x0002b44ddb1476db43c868bd494422ee4c136fed,0,12709.07,2958.44,1218216.73,94,8,0,...,2.260809,2.260809e+00,2.260809,0.0,0.0,0.0,1.0,7.0,Livepeer Token,Livepeer Token
2,2,3,0x0002bda54cb772d040f779e88eb453cac0daa244,0,246194.54,2434.02,516729.30,2,10,0,...,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,8.0,None,XENON
3,3,4,0x00038e6ba2fd5c09aedb96697c8d7b8fa6632e5e,0,10219.60,15785.09,397555.90,25,9,0,...,100.000000,9.029231e+03,3804.076893,0.0,0.0,0.0,1.0,11.0,Raiden,XENON
4,4,5,0x00062d1dd1afb6fb02540ddad9cdebfe568e0d89,0,36.61,10707.77,382472.42,4598,20,1,...,0.000000,4.500000e+04,13726.659220,0.0,0.0,0.0,6.0,27.0,StatusNetwork,EOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9836,9836,2175,0xff481ca14e6c16b79fc8ab299b4d2387ec8ecdd2,1,12635.10,631.39,58748.48,4,13,0,...,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,2.0,,GSENetwork
9837,9837,2176,0xff718805bb9199ebf024ab6acd333e603ad77c85,1,0.00,0.00,0.00,0,0,0,...,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,1.0,,Blockwell say NOTSAFU
9838,9838,2177,0xff8e6af02d41a576a0c82f7835535193e1a6bccc,1,2499.44,2189.29,261601.88,67,43,0,...,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,5.0,,Free BOB Tokens - BobsRepair.com
9839,9839,2178,0xffde23396d57e10abf58bd929bb1e856c7718218,1,0.00,0.00,0.00,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(9841, 51)

In [5]:
df['FLAG'].value_counts()

0    7662
1    2179
Name: FLAG, dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9841 entries, 0 to 9840
Data columns (total 51 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Unnamed: 0                                            9841 non-null   int64  
 1   Index                                                 9841 non-null   int64  
 2   Address                                               9841 non-null   object 
 3   FLAG                                                  9841 non-null   int64  
 4   Avg min between sent tnx                              9841 non-null   float64
 5   Avg min between received tnx                          9841 non-null   float64
 6   Time Diff between first and last (Mins)               9841 non-null   float64
 7   Sent tnx                                              9841 non-null   int64  
 8   Received Tnx                                          9841

# Engenharia de Atributos


Vamos limpar as variaveis para utilizar em nosso modelo 

In [7]:
df.columns

Index(['Unnamed: 0', 'Index', 'Address', 'FLAG', 'Avg min between sent tnx',
       'Avg min between received tnx',
       'Time Diff between first and last (Mins)', 'Sent tnx', 'Received Tnx',
       'Number of Created Contracts', 'Unique Received From Addresses',
       'Unique Sent To Addresses', 'min value received', 'max value received ',
       'avg val received', 'min val sent', 'max val sent', 'avg val sent',
       'min value sent to contract', 'max val sent to contract',
       'avg value sent to contract',
       'total transactions (including tnx to create contract',
       'total Ether sent', 'total ether received',
       'total ether sent contracts', 'total ether balance',
       ' Total ERC20 tnxs', ' ERC20 total Ether received',
       ' ERC20 total ether sent', ' ERC20 total Ether sent contract',
       ' ERC20 uniq sent addr', ' ERC20 uniq rec addr',
       ' ERC20 uniq sent addr.1', ' ERC20 uniq rec contract addr',
       ' ERC20 avg time between sent tnx', ' ERC20 

In [8]:
df.columns = df.columns.str.strip()

In [9]:
df.columns = [x.lower() for x in df.columns]
df.columns

Index(['unnamed: 0', 'index', 'address', 'flag', 'avg min between sent tnx',
       'avg min between received tnx',
       'time diff between first and last (mins)', 'sent tnx', 'received tnx',
       'number of created contracts', 'unique received from addresses',
       'unique sent to addresses', 'min value received', 'max value received',
       'avg val received', 'min val sent', 'max val sent', 'avg val sent',
       'min value sent to contract', 'max val sent to contract',
       'avg value sent to contract',
       'total transactions (including tnx to create contract',
       'total ether sent', 'total ether received',
       'total ether sent contracts', 'total ether balance', 'total erc20 tnxs',
       'erc20 total ether received', 'erc20 total ether sent',
       'erc20 total ether sent contract', 'erc20 uniq sent addr',
       'erc20 uniq rec addr', 'erc20 uniq sent addr.1',
       'erc20 uniq rec contract addr', 'erc20 avg time between sent tnx',
       'erc20 avg time be

In [10]:
cols_drop = ['erc20 most sent token type',
                'erc20_most_rec_token_type',
                'address',
                'index',
                'unnamed: 0']

In [11]:
atributos =  [x for x in df.columns if x != 'flag' and  x not in cols_drop]

In [12]:
df.columns

Index(['unnamed: 0', 'index', 'address', 'flag', 'avg min between sent tnx',
       'avg min between received tnx',
       'time diff between first and last (mins)', 'sent tnx', 'received tnx',
       'number of created contracts', 'unique received from addresses',
       'unique sent to addresses', 'min value received', 'max value received',
       'avg val received', 'min val sent', 'max val sent', 'avg val sent',
       'min value sent to contract', 'max val sent to contract',
       'avg value sent to contract',
       'total transactions (including tnx to create contract',
       'total ether sent', 'total ether received',
       'total ether sent contracts', 'total ether balance', 'total erc20 tnxs',
       'erc20 total ether received', 'erc20 total ether sent',
       'erc20 total ether sent contract', 'erc20 uniq sent addr',
       'erc20 uniq rec addr', 'erc20 uniq sent addr.1',
       'erc20 uniq rec contract addr', 'erc20 avg time between sent tnx',
       'erc20 avg time be

In [13]:
atributos

['avg min between sent tnx',
 'avg min between received tnx',
 'time diff between first and last (mins)',
 'sent tnx',
 'received tnx',
 'number of created contracts',
 'unique received from addresses',
 'unique sent to addresses',
 'min value received',
 'max value received',
 'avg val received',
 'min val sent',
 'max val sent',
 'avg val sent',
 'min value sent to contract',
 'max val sent to contract',
 'avg value sent to contract',
 'total transactions (including tnx to create contract',
 'total ether sent',
 'total ether received',
 'total ether sent contracts',
 'total ether balance',
 'total erc20 tnxs',
 'erc20 total ether received',
 'erc20 total ether sent',
 'erc20 total ether sent contract',
 'erc20 uniq sent addr',
 'erc20 uniq rec addr',
 'erc20 uniq sent addr.1',
 'erc20 uniq rec contract addr',
 'erc20 avg time between sent tnx',
 'erc20 avg time between rec tnx',
 'erc20 avg time between rec 2 tnx',
 'erc20 avg time between contract tnx',
 'erc20 min val rec',
 'erc20

In [14]:
valores_unicos = df.nunique()
len(valores_unicos)

51

In [15]:
atributos = [x for x in atributos if x in valores_unicos.loc[valores_unicos > 1]]
atributos

['avg min between sent tnx',
 'avg min between received tnx',
 'time diff between first and last (mins)',
 'sent tnx',
 'received tnx',
 'number of created contracts',
 'unique received from addresses',
 'unique sent to addresses',
 'min value received',
 'max value received',
 'avg val received',
 'min val sent',
 'max val sent',
 'avg val sent',
 'min value sent to contract',
 'max val sent to contract',
 'avg value sent to contract',
 'total transactions (including tnx to create contract',
 'total ether sent',
 'total ether received',
 'total ether sent contracts',
 'total ether balance',
 'total erc20 tnxs',
 'erc20 total ether received',
 'erc20 total ether sent',
 'erc20 total ether sent contract',
 'erc20 uniq sent addr',
 'erc20 uniq rec addr',
 'erc20 uniq sent addr.1',
 'erc20 uniq rec contract addr',
 'erc20 min val rec',
 'erc20 max val rec',
 'erc20 avg val rec',
 'erc20 min val sent',
 'erc20 max val sent',
 'erc20 avg val sent',
 'erc20 uniq sent token name',
 'erc20 uni

In [16]:
atributos =  [x for x in df.columns if x != 'flag' and  x not in cols_drop]
len(atributos)

45

In [17]:
atributos = [x for x in atributos if x in valores_unicos.loc[valores_unicos > 1]]
len(atributos)

38

In [18]:
df[atributos].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9841 entries, 0 to 9840
Data columns (total 38 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   avg min between sent tnx                              9841 non-null   float64
 1   avg min between received tnx                          9841 non-null   float64
 2   time diff between first and last (mins)               9841 non-null   float64
 3   sent tnx                                              9841 non-null   int64  
 4   received tnx                                          9841 non-null   int64  
 5   number of created contracts                           9841 non-null   int64  
 6   unique received from addresses                        9841 non-null   int64  
 7   unique sent to addresses                              9841 non-null   int64  
 8   min value received                                    9841

In [19]:
df_novo = df[atributos]

In [20]:
df_novo.columns = df_novo.columns.str.strip()

In [21]:
df_novo.head()

,avg min between sent tnx,avg min between received tnx,time diff between first and last (mins),sent tnx,received tnx,number of created contracts,unique received from addresses,unique sent to addresses,min value received,max value received,...,erc20 uniq sent addr.1,erc20 uniq rec contract addr,erc20 min val rec,erc20 max val rec,erc20 avg val rec,erc20 min val sent,erc20 max val sent,erc20 avg val sent,erc20 uniq sent token name,erc20 uniq rec token name
0,844.26,1093.71,704785.63,721,89,0,40,118,0.000000,45.806785,...,0.0,58.0,0.0,1.500000e+07,265586.147600,0.000000,1.683100e+07,271779.920000,39.0,57.0
1,12709.07,2958.44,1218216.73,94,8,0,5,14,0.000000,2.613269,...,0.0,7.0,0.0,3.650000e+02,57.632615,2.260809,2.260809e+00,2.260809,1.0,7.0
2,246194.54,2434.02,516729.30,2,10,0,10,2,0.113119,1.165453,...,0.0,8.0,0.0,4.428198e+02,65.189009,0.000000,0.000000e+00,0.000000,0.0,8.0
3,10219.60,15785.09,397555.90,25,9,0,7,13,0.000000,500.000000,...,0.0,11.0,0.0,1.141223e+04,1555.550174,100.000000,9.029231e+03,3804.076893,1.0,11.0
4,36.61,10707.77,382472.42,4598,20,1,7,19,0.000000,12.802411,...,0.0,27.0,0.0,9.000000e+04,4934.232147,0.000000,4.500000e+04,13726.659220,6.0,27.0


# Criando o Pipeline
### Pipeline tem como objetivo facilitar o tratamento dos novos dados no Deploy

Criarei 4 Classes para utilziar no pipeline.

1. Classe PipeSteps:

Classe base para transformar dados em um pipeline. Serve como um esqueleto com métodos **fit** e **transform**, permitindo personalizações nas subclasses.

2. Classe SelecionaColunas:

Herda de PipeSteps e implementa o método transform para selecionar apenas as colunas especificadas na lista self.columns.
- Entrada: Um DataFrame.
- Saída: DataFrame contendo apenas as colunas selecionadas.
- Uso: Foco em trabalhar apenas com as colunas relevantes para o pipeline.

3. PreenchaDados:

Herda de PipeSteps e implementa lógica para preencher valores ausentes (NaN) nas colunas especificadas.
- fit: Calcula a média de cada coluna especificada e armazena em self.means.
- transform: Preenche os valores ausentes em cada coluna com a média calculada.
- Uso: Garante que as colunas não tenham valores ausentes antes de outras transformações.


4. PadronizaDados: 

Herda de PipeSteps e usa o StandardScaler para padronizar os dados das colunas especificadas (média 0, desvio padrão 1).
- fit: Ajusta o StandardScaler com base nos dados das colunas.
- transform: Aplica a transformação de padronização nas colunas.
- Uso: Normaliza as colunas para facilitar o uso em algoritmos sensíveis a escalas.



In [22]:
class PipeSteps(BaseEstimator , TransformerMixin):

    # Método contrutor para iniciar a classe com uma lista de columas 
    def __init__(self, columns=[]):
        self.columns = columns 

    # Método utilizado para treinar a transformação dos dados de treinamento
    def fit(self, X, y = None):
        return self
    
    # Método para transformar os dados de entrada
    def transform(self, X):
        # cópia dos dados de entrada para evitar alterar os dados originais
        X = X.copy()
        return X
        

In [23]:
class SelecionaColunas(PipeSteps):

    def transform(self, X): 
        X = X.copy()
        return X[self.columns]

In [24]:
class PreencheDados(PipeSteps):

    def fit(self, X, y=None):

        # Calcula a média de cada coluna especificada em self.columns e armazena no dicionário self.means 
        self.means =  { col: X[col].mean() for col in self.columns }
        return self
    
    def transform(self, X):
        X = X.copy()

        for col in self.columns:
            X[col] = X[col].fillna(self.means[col])
        
        return X



In [25]:
class PadronizaDados(PipeSteps):

    # Método para padronizar os dados
    def fit(self, X, y = None):
        self.scater = StandardScaler()
        self.scater.fit(X[self.columns])
        return self  
    
    def transform(self, X):
        X = X.copy()
        X[self.columns] = self.scater.transform(X[self.columns])
        return X

In [26]:
# Cria o pipeline de pré-processamento
pipe_prepropcessamento = Pipeline([('feature_selection', SelecionaColunas(atributos)),
                                    ('fill_missing', PreencheDados(atributos)),
                                    ('standard_scaling', PadronizaDados(atributos))])

In [27]:
pipe_prepropcessamento

Pipeline(steps=[('feature_selection',
                 SelecionaColunas(columns=['avg min between sent tnx',
                                           'avg min between received tnx',
                                           'time diff between first and last '
                                           '(mins)',
                                           'sent tnx', 'received tnx',
                                           'number of created contracts',
                                           'unique received from addresses',
                                           'unique sent to addresses',
                                           'min value received',
                                           'max value received',
                                           'avg val received', 'min val sent',
                                           'max val sent', 'avg val sent',...
                                         'avg value sent to contract',
                                 

## Criação do Pipeline de Machine Learning

In [28]:
pipe_final = Pipeline([
    ('preprocessing', pipe_prepropcessamento),
    ('learning', XGBClassifier(random_state = 42, eval_metric = 'auc', objective = 'binary:logistic') )
])

# Preparação dos dados 

In [29]:
X = df_novo[atributos]



In [30]:
y = df['flag']

In [31]:
X_treino, X_teste , y_treino, y_teste = train_test_split(X, y , test_size=0.3, random_state=42)

In [32]:
X_treino

,avg min between sent tnx,avg min between received tnx,time diff between first and last (mins),sent tnx,received tnx,number of created contracts,unique received from addresses,unique sent to addresses,min value received,max value received,...,erc20 uniq sent addr.1,erc20 uniq rec contract addr,erc20 min val rec,erc20 max val rec,erc20 avg val rec,erc20 min val sent,erc20 max val sent,erc20 avg val sent,erc20 uniq sent token name,erc20 uniq rec token name
8460,0.00,0.00,0.00,0,0,0,0,0,0.000000,0.000000,...,0.0,1.0,1.337000,1.337000,1.337000,0.0,0.0,0.0,0.0,1.0
6081,0.00,8337.42,200098.17,0,24,1,3,0,0.000000,1.003651,...,0.0,2.0,0.301638,0.953298,0.627468,0.0,0.0,0.0,0.0,2.0
8966,0.00,318.17,2287.65,1,2,0,2,1,0.005000,0.300000,...,0.0,1.0,20000.000000,542000.000000,281000.000000,0.0,0.0,0.0,0.0,1.0
1535,28.33,3901.86,1021678.35,254,260,0,25,2,0.000047,110.736000,...,0.0,5.0,0.000000,19421.000000,3961.659270,0.0,0.0,0.0,0.0,5.0
7304,0.00,1947.10,319323.80,0,164,1,4,0,0.000000,1.903509,...,0.0,3.0,0.000000,2.335693,1.313279,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,0.00,16197.17,226760.43,0,14,1,6,0,0.000000,14.419115,...,0.0,2.0,0.518689,0.705159,0.611924,0.0,0.0,0.0,0.0,2.0
5191,0.00,0.00,15369.12,1,1,0,1,1,2.000000,2.000000,...,0.0,7.0,0.000000,312.430205,50.497598,0.0,0.0,0.0,0.0,7.0
5390,0.00,0.00,1.77,1,1,0,1,1,1.990000,1.990000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
860,165.15,0.00,330.30,2,2,0,2,2,49.770407,51.229593,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [33]:
X_teste

,avg min between sent tnx,avg min between received tnx,time diff between first and last (mins),sent tnx,received tnx,number of created contracts,unique received from addresses,unique sent to addresses,min value received,max value received,...,erc20 uniq sent addr.1,erc20 uniq rec contract addr,erc20 min val rec,erc20 max val rec,erc20 avg val rec,erc20 min val sent,erc20 max val sent,erc20 avg val sent,erc20 uniq sent token name,erc20 uniq rec token name
7763,1641.74,2103.12,327679.35,10,148,0,137,4,0.001000,14.341000,...,0.0,5.0,0.00,2082.268173,419.127635,0.0,0.0,0.0,0.0,5.0
8655,2811.51,837.98,9812.92,2,5,0,4,2,0.490000,0.500000,...,0.0,1.0,13.37,13.370000,13.370000,0.0,0.0,0.0,0.0,1.0
106,157.32,0.00,314.65,2,1,0,1,2,101.000000,101.000000,...,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
439,20.17,3.92,68.37,3,2,0,2,3,12.690172,88.309828,...,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2420,4.38,24303.06,243074.38,10,10,0,2,1,0.574000,9.000000,...,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3574,162.63,0.00,325.27,2,1,0,1,2,101.000000,101.000000,...,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4824,7192.69,58549.99,204420.75,4,3,0,2,1,5.013159,40.391000,...,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1157,1.77,0.41,6.12,3,2,0,2,3,434.001122,1566.998878,...,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
7786,0.00,6.27,1310.22,0,209,1,6,0,0.000000,2.000000,...,0.0,1.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0


## Treinamento e Avaliação do Modelo

In [34]:
# Treino do modelo
pipe_final.fit(X_treino, y_treino)

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('feature_selection',
                                  SelecionaColunas(columns=['avg min between '
                                                            'sent tnx',
                                                            'avg min between '
                                                            'received tnx',
                                                            'time diff between '
                                                            'first and last '
                                                            '(mins)',
                                                            'sent tnx',
                                                            'received tnx',
                                                            'number of created '
                                                            'contracts',
                                                            'unique receiv

In [35]:
# Previsões com dados de teste
previsoes_teste = pipe_final.predict(X_teste)

In [36]:
previsoes_teste

array([1, 1, 0, ..., 0, 1, 0])

In [37]:
score_auc = metrics.roc_auc_score(y_teste, previsoes_teste)
print(f'AUC nos Dados de Teste - {score_auc:,.2%}')

AUC nos Dados de Teste - 97.27%


In [38]:
# Salva o modelo em disco
dump(pipe_final, 'modelo_final.joblib')

['modelo_final.joblib']

## Deploy do Modelo e Detecção de Fraudes em Novas Transações com Criptomoedas

In [39]:
# Carrega os novos dados de uma transação
novos_dados = pd.read_csv('novos_dados.csv')

In [40]:
novos_dados

,avg min between sent tnx,avg min between received tnx,time diff between first and last (mins),sent tnx,received tnx,number of created contracts,unique received from addresses,unique sent to addresses,min value received,max value received,...,erc20 uniq sent addr.1,erc20 uniq rec contract addr,erc20 min val rec,erc20 max val rec,erc20 avg val rec,erc20 min val sent,erc20 max val sent,erc20 avg val sent,erc20 uniq sent token name,erc20 uniq rec token name
0,2570.59,3336.01,30572.7,8,3,0,2,4,0.1,40.0,...,0.0,1.0,600.0,600.0,600.0,0.0,0.0,0.0,0.0,1.0


In [41]:
# Carrega o modelo do disco
modelo_carregado = load('modelo_final.joblib')

In [42]:
novos_dados.columns = novos_dados.columns.str.strip()

In [43]:
previsao = modelo_carregado.predict(novos_dados)

In [44]:
previsao

array([0])

In [45]:
# Resultado
if previsao[0] == 0:
    print("Segundo o modelo, provavelmente, essa transação não representa uma Fraude.")
else:
    print("Segundo o modelo, provavelmente, essa transação pode representar uma Fraude. Acione verificação humana!")

Segundo o modelo, provavelmente, essa transação não representa uma Fraude.
